In [1]:
from gradio import Interface, Audio
import gradio as gr
import numpy as np
import torch
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import os
from functools import partial
import gin
import torchaudio
from absl import app
from torch.nn.functional import interpolate
import pdb
import logging
import time
import soundfile as sf

/home/mila/k/krishna-maneesha.dendukuri/.conda/envs/indian_improv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append("/home/mila/k/krishna-maneesha.dendukuri")
sys.path.append("/home/mila/k/krishna-maneesha.dendukuri/GaMaDHaNi")
sys.path.append("/home/mila/k/krishna-maneesha.dendukuri/GaMaDHaNi/src")
import GaMaDHaNi
from GaMaDHaNi import src, utils, scripts
# from GaMaDHaNi.src import protobuf
from src.protobuf.data_example import AudioExample
from src.dataset import Task
from utils.generate_utils import load_pitch_model, load_audio_model
import utils.pitch_to_audio_utils as p2a
from utils.utils import get_device, plot, save_figure, save_csv, save_audio
sys.path.append(os.path.abspath("../scripts"))
from generate import load_pitch_fns, load_audio_fns, generate_pitch, generate_audio


In [3]:
global_ind = -1
global_audios = np.array([0.0])
global_pitches = np.array([0])
singer = 3
audio_components = []
preprocessed_primes = []
selected_prime = None
pitch_model_type = 'transformer'
number_of_samples = 2

In [4]:
pitch_path = '/home/mila/k/krishna-maneesha.dendukuri/scratch/checkpoints/msprior/x-transformer-debug/4340302'
audio_path = '/network/scratch/n/nithya.shikarpur/checkpoints/pitch-diffusion/corrected-attention-v3/4835364'
prime_file = '/home/mila/k/krishna-maneesha.dendukuri/notebooks/listening_study_primes.npz'
db_path_audio = '/network/scratch/n/nithya.shikarpur/pitch-diffusion/data/merged_data-finalest/cached-audio-pitch-16k'
output_folder = '/home/mila/k/krishna-maneesha.dendukuri/scratch/generations/demo_v1'

In [5]:
def generate(audio_model: str=None, 
            pitch_model: str=None, 
            pitch_model_type: str=None,
            num_samples: int=1, 
            invert_pitch_fn=None,
            invert_audio_fn=None,
            num_steps: int=100, 
            singers: list=[3], 
            seq_len=1200,
            temperature=1., 
            outfolder: str='temp', 
            audio_seq_len: int=750, 
            pitch_qt=None,
            processed_primes=None, 
            prime=False,
            ):
    global global_ind, audio_components
    global preprocessed_primes
    global_ind += 1
    logging.log(logging.INFO, 'Generate function')
    pdb.set_trace()
    pitch, inverted_pitch = generate_pitch(pitch_model=pitch_model,
                                           pitch_model_type=pitch_model_type,
                                           invert_pitch_fn=invert_pitch_fn, 
                                           num_samples=num_samples, 
                                           seq_len=seq_len,
                                           temperature=temperature,
                                           num_steps=num_steps, 
                                           outfolder=outfolder, 
                                           processed_primes=selected_prime if global_ind != 0 else None,
                                           pitch_sample_rate=200,
                                           prime=prime)
    preprocessed_primes = pitch[:, :, 200:400]
    if pitch_qt is not None:
        def undo_qt(x, min_clip=200):
            pitch= pitch_qt.inverse_transform(x.reshape(-1, 1)).reshape(1, -1)
            pitch = np.around(pitch) # round to nearest integer, done in preprocessing of pitch contour fed into model
            pitch[pitch < 200] = np.nan
            return pitch
        pitch = torch.tensor(np.array([undo_qt(x) for x in pitch.detach().cpu().numpy()])).to(pitch_model.device)
    interpolated_pitch = p2a.interpolate_pitch(pitch=pitch, audio_seq_len=audio_seq_len)
    interpolated_pitch = torch.nan_to_num(interpolated_pitch, nan=196)
    interpolated_pitch = interpolated_pitch.squeeze(1) # to match input size by removing the extra dimension
    audio = generate_audio(audio_model, 
                           interpolated_pitch, 
                           invert_audio_fn, 
                           singers=singers, 
                           num_steps=100, 
                           outfolder=outfolder)[:, :16000*4]
    # pdb.set_trace()
    audio = audio.detach().cpu().numpy()
    state = [(16000, audio[0]), (16000, audio[1])]
    pitch_vals = [np.where(inverted_pitch[0][:, 0] == 0, np.nan, inverted_pitch[0].flatten())[:400], np.where(inverted_pitch[1][:, 0] == 0, np.nan, inverted_pitch[1].flatten())[:400]]
    fig1 = plt.figure()
    plt.plot(pitch_vals[0], figure=fig1)
    fig2 = plt.figure()
    plt.plot(pitch_vals[1], figure=fig2)
    state.append(fig1)
    plt.close(fig1)
    state.append(fig2)
    plt.close(fig2)
    state.append(state[0:2]) #this has to be fixed to be more elegant
    state.append(pitch_vals)
    return state

In [8]:
def clear_fn():
    global global_ind, global_audios, global_pitches
    global_ind = -1
    global_audios = np.array([0.0])
    global_pitches = np.array([0])
    cmd = f"rm -rf ./temp && mkdir ./temp"
    subprocess.run(cmd, shell=True)


def update_choice(selected_options, full_audio, full_pitch, option, audiostate, pitchstate):
    global global_ind
    option = int(option)
    selected_options.append({"audio": audiostate[option]})
    full_audio[1] = np.concatenate((full_audio[1], audiostate[option][1][(2)*16000*int(global_ind!=0):]))
    full_pitch = np.concatenate((full_pitch, pitchstate[option][200*int(global_ind!=0):]))
    
    global selected_prime, preprocessed_primes
    selected_prime = preprocessed_primes[option].unsqueeze(0)

    fig = plt.figure()
    plt.plot(full_pitch, figure=fig)
    plt.close(fig)

    return (16000, full_audio[1]), fig, full_audio, full_pitch

def save_session(full_pitch, full_audio):
    os.makedirs(output_folder, exist_ok=True)
    filename = f'session-{time.time()}'
    logging.log(logging.INFO, f"Saving session to {filename}")
    pd.DataFrame({'pitch': full_pitch, 'time': np.arange(0, len(full_pitch)/100, 0.01)}).to_csv(os.path.join(output_folder, filename + '.csv'), index=False)
    sf.write(os.path.join(output_folder, filename + '.wav'), full_audio[1], 16000)

def create_gradio_interface():
    with gr.Blocks() as demo:
        selected_options = gr.State([])
        full_audio = gr.State((16000, np.array([])))
        full_pitch = gr.State(np.array([]))
        generated_audios = gr.State([(16000, []), (16000, [])])
        generated_pitches = gr.State([None, None])

        btn1 = gr.Button("Create Some Music!!")
        btn2 = gr.Button("Save Session")
        btn2.click(fn=save_session, inputs=[full_pitch, full_audio])

        for i in range(10):
            gr.Markdown(f"### Generation {i+1}")
            audio_components.append(create_audio_components(visible=True))
            audio_components[i]['submit'].click(fn=update_choice, 
                inputs=[selected_options, full_audio, full_pitch, audio_components[i]['dropdown'], generated_audios, generated_pitches], 
                outputs=[audio_components[i]['generated_audio'], audio_components[i]['generated_pitch'], full_audio, full_pitch])

        for i in range(9):
            audio_components[i]['generate_next'].click(fn=partial_generate, 
                inputs=[generated_audios, generated_pitches], 
                outputs=[audio_components[i+1]['option 0'], audio_components[i+1]['option 1'], 
                         audio_components[i+1]['pitch 0'], audio_components[i+1]['pitch 1'], 
                         generated_audios, generated_pitches])

        btn1.click(fn=partial_generate, 
            inputs=[generated_audios, generated_pitches], 
            outputs=[audio_components[0]['option 0'], audio_components[0]['option 1'], 
                     audio_components[0]['pitch 0'], audio_components[0]['pitch 1'], 
                     generated_audios, generated_pitches])

    return demo

pitch model loaded on the device: cuda:0
audio model loaded on the device: cuda:0


/home/mila/k/krishna-maneesha.dendukuri/.conda/envs/indian_improv/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator QuantileTransformer from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    pitch_model, pitch_qt, pitch_task_fn, invert_pitch_fn, primes = load_pitch_fns(pitch_path=pitch_path, 
                                                                                model_type = pitch_model_type, 
                                                                                prime=True, 
                                                                                prime_file=prime_file, 
                                                                                number_of_samples=number_of_samples)  
    logging.info(f"Pitch model loaded on the device: {pitch_model.device}")
    audio_model, audio_qt, audio_seq_len, invert_audio_fn = load_audio_fns(audio_path=audio_path, 
                                                                            db_path_audio=db_path_audio)
    logging.info(f"Audio model loaded on the device: {audio_model.device}")
    partial_generate = partial(generate, num_samples=2, num_steps=100, singers=[3], outfolder='temp', pitch_qt=pitch_qt)

    demo = create_gradio_interface()
    demo.launch(show_example=False)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f53ff0cfba68c02b44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


> /tmp/ipykernel_1034456/3961911109.py(22)generate()
     20     logging.log(logging.INFO, 'Generate function')
     21     pdb.set_trace()
---> 22     pitch, inverted_pitch = generate_pitch(pitch_model=pitch_model,
     23                                            pitch_model_type=pitch_model_type,
     24                                            invert_pitch_fn=invert_pitch_fn,

ipdb> 

In [7]:
import gradio
gradio.__version__

'4.41.0'